# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#import libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, ne_chunk

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# import libraries
import re
import numpy as np
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
import pandas as pd
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle
#from sklearn.ensemble import AdaBoostRegressor
#from sklearn.cluster import KMeans
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import SGDClassifier

In [56]:
#check sqlalchemy
import sqlalchemy
print (sqlalchemy.__version__ )

1.1.13


In [73]:
#load data from database
def load_data():
    engine = create_engine('sqlite:///disaster_response.db')
    df = pd.read_sql("SELECT * FROM disaster_response", engine)
    X = df.message.values
    #exclude id, message and genre from y variable
    y=df.iloc[:,3:]
    return X,y

In [6]:
#inspect first rows
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
#call function
X,y = load_data()

In [7]:
#shape of our dataset
X.shape, y.shape

((26216,), (26216, 36))

In [4]:
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM disaster_response", engine)

In [5]:
# find top 5 and least 5 categories
df= df.replace(2,1)
df = df.iloc[:,3:]

In [11]:
#count number of true values
count_val = (df.apply(pd.value_counts).transpose()).iloc[:, 1:2]

In [14]:
count_val['count'] = count_val.iloc[:,0]

In [19]:
#count_val.sort_index(ascending=False)
top_five = count_val.sort_values(by=['count'], ascending=False)[0:5]
top_freq = top_five['count']

In [21]:
#list of top categories
top_names = list(top_freq.index)

['related', 'aid_related', 'weather_related', 'direct_report', 'request']

In [23]:
#obtain least 5 categories
least_five = count_val.sort_values(by=['count'], ascending=True)[0:5]
least_freq = least_five['count']
least_names = list(least_freq.index)

In [26]:
#the five least frequencies
least_freq

offer        118.0
shops        120.0
tools        159.0
fire         282.0
hospitals    283.0
Name: count, dtype: float64

### 2. Write a tokenization function to process your text data

In [35]:
def tokenize(text):
    """
    INPUT
    text: text to be stemmed and lemmatized
    OUTPUT:
    clean_tokens: text after remove stopwords, reduce to their stems,
                    lematize and normalize 
    """
    #Normalize text
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    #tokenize text
    tokens = word_tokenize(text)
    #remove stop words
    tokens_stop = [w for w in tokens if w not in stopwords.words("english")]
    #reduce words to their stems
    tokens_stem = [PorterStemmer().stem(w) for w in tokens_stop]
    #initiate lemmatizer 
    lemmatizer = WordNetLemmatizer()
    #iterate through each token
    clean_tokens=[]
    for tok in tokens_stem:
        #lemmatize, normalize case and remove white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [11]:
#test function:
#for message in X[:11]:
#    tokens=tokenize(message)
#    print (message)
#    print(tokens)

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over
['hurrican']
Looking for someone but no name
['look', 'someon', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight']
Information about the National Palace-
['inform', 'nation', 'palac']
Storm at sacred heart of jesus
['storm', 'sacr', 'heart', 'jesu']
Please, we need tents and water. We are in Silo, Thank you!
['plea', 'need', 'tent', 'water', 'silo', 'thank']
I would like to receive the messages, thank you
['would', 'like', 'receiv', 'messag', 'thank']
I am in Croix-des-Bouquets. We have health issues. They (

In [6]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [6]:
#apply CountVectorizer
class apply_fit_transform(BaseEstimator, TransformerMixin):
    def vect(self, text):
        vec = CountVectorizer(tokenizer=tokenize)
        X=vec.fit_transform(X_train)
    def fit(self, x, y=None):
        return self
    def Transform(self, X):
        X_tr = pd.Series(X).apply(self.vect)
        return pd.Dataframe(X_tr)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### Try with KNeighborsClassifier

In [23]:
#build pipeline 
#using KNeighborsClassifier
pipeline=Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(KNeighborsClassifier()))
])


In [24]:
#get parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f65e4052d08>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_fea

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [26]:
#fit pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [28]:
#predict on test data
y_pred = pipeline.predict(X_test)

In [30]:
#find accuracy
accuracy = (y_pred == y_test).mean()
print(accuracy)

related                   0.778640
request                   0.842848
offer                     0.995423
aid_related               0.601399
medical_help              0.918754
medical_products          0.946599
search_and_rescue         0.975588
security                  0.981564
military                  0.968849
child_alone               1.000000
water                     0.938207
food                      0.893198
shelter                   0.913160
clothing                  0.985887
money                     0.978385
missing_people            0.988303
refugees                  0.966942
death                     0.954100
other_aid                 0.867387
infrastructure_related    0.935664
transport                 0.953973
buildings                 0.950540
electricity               0.978640
tools                     0.993897
hospitals                 0.990083
shops                     0.996440
aid_centers               0.986904
other_infrastructure      0.956516
weather_related     

#### Try Random forest classifier

In [16]:
#neww pipeline: trying random forest classifier
pipeline=Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

In [33]:
#get parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f65e4052d08>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [17]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [35]:
#predicting
y_pred = pipeline.predict(X_test)

In [36]:
#finf accuracy
accuracy = (y_pred == y_test).mean()
print(accuracy)

related                   0.802670
request                   0.884806
offer                     0.995423
aid_related               0.750540
medical_help              0.921933
medical_products          0.950413
search_and_rescue         0.975588
security                  0.981437
military                  0.969739
child_alone               1.000000
water                     0.948887
food                      0.932231
shelter                   0.929943
clothing                  0.987285
money                     0.979148
missing_people            0.988303
refugees                  0.966052
death                     0.957788
other_aid                 0.868913
infrastructure_related    0.935029
transport                 0.954736
buildings                 0.953973
electricity               0.978767
tools                     0.993897
hospitals                 0.990083
shops                     0.996440
aid_centers               0.986904
other_infrastructure      0.955880
weather_related     

#### RandomForestClassifier' output with higher accuracy when compared with KNeighborsClassifier

In [18]:
#pipeline using FeatureUnion
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
    ])),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

In [12]:
#get parameters
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              

In [19]:
#fit pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [20]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
#find shape
y_pred.shape,y_test.shape,len(list(y.columns))

((7865, 36), (7865, 36), 36)

In [21]:
#make names of columns equal in y_pred and y_test
y_pred = pd.DataFrame(y_pred, columns = y.columns)
y_test = pd.DataFrame(y_test, columns=y.columns)

In [24]:
#see columns
y_pred.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [21]:
#see columns
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [22]:
#replace all 2 with 1 in order to avoid errors
y_test = y_test.replace(2,1)

In [44]:
#verify values
print(np.unique(y_test))

[0 1]


In [23]:
#replace 2 with 1 to avoid error
y_pred = y_pred.replace(2,1)

In [25]:
#print classification report
target_names=y.columns
print(classification_report(y_test, y_pred, target_names = target_names))

                        precision    recall  f1-score   support

               related       0.84      0.91      0.88      5992
               request       0.77      0.40      0.52      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.73      0.61      0.66      3219
          medical_help       0.58      0.10      0.17       638
      medical_products       0.75      0.07      0.13       418
     search_and_rescue       0.40      0.02      0.04       192
              security       0.00      0.00      0.00       144
              military       0.61      0.08      0.14       245
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.27      0.41       500
                  food       0.85      0.47      0.60       878
               shelter       0.78      0.28      0.41       705
              clothing       0.82      0.27      0.41       115
                 money       0.69      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
#listing parameters
list(pipeline.get_params().keys())

['memory',
 'steps',
 'features',
 'clf',
 'features__n_jobs',
 'features__transformer_list',
 'features__transformer_weights',
 'features__text_pipeline',
 'features__text_pipeline__memory',
 'features__text_pipeline__steps',
 'features__text_pipeline__vect',
 'features__text_pipeline__tfidf',
 'features__text_pipeline__vect__analyzer',
 'features__text_pipeline__vect__binary',
 'features__text_pipeline__vect__decode_error',
 'features__text_pipeline__vect__dtype',
 'features__text_pipeline__vect__encoding',
 'features__text_pipeline__vect__input',
 'features__text_pipeline__vect__lowercase',
 'features__text_pipeline__vect__max_df',
 'features__text_pipeline__vect__max_features',
 'features__text_pipeline__vect__min_df',
 'features__text_pipeline__vect__ngram_range',
 'features__text_pipeline__vect__preprocessor',
 'features__text_pipeline__vect__stop_words',
 'features__text_pipeline__vect__strip_accents',
 'features__text_pipeline__vect__token_pattern',
 'features__text_pipeline__v

In [32]:
#choose parameters
parameters = {
    'clf__estimator__n_estimators':[100, 150, 200],
}
# use GridSearchCV
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [33]:
#fit model
model=cv.fit(X_train, y_train)


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.25862350825568087, total= 6.3min
[CV] clf__estimator__n_estimators=100 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, score=0.26745136504822625, total= 6.3min
[CV] clf__estimator__n_estimators=100 ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 15.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, score=0.2694131110021252, total= 6.3min
[CV] clf__estimator__n_estimators=150 ................................
[CV]  clf__estimator__n_estimators=150, score=0.2574791564492398, total= 8.6min
[CV] clf__estimator__n_estimators=150 ................................
[CV]  clf__estimator__n_estimators=150, score=0.2651626614353441, total= 8.7min
[CV] clf__estimator__n_estimators=150 ................................
[CV]  clf__estimator__n_estimators=150, score=0.26467222494686937, total= 8.6min
[CV] clf__estimator__n_estimators=200 ................................
[CV]  clf__estimator__n_estimators=200, score=0.2582965505966977, total=11.1min
[CV] clf__estimator__n_estimators=200 ................................
[CV]  clf__estimator__n_estimators=200, score=0.26483570377636095, total=11.1min
[CV] clf__estimator__n_estimators=200 ................................
[CV]  clf__estimator__n_estimators=200, score=0.26745136504822625, total=11.1min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 93.9min finished


In [34]:
#predict
y_pred=model.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
#find accuracy
accuracy = (y_pred == y_test).mean()
print(accuracy)

related                   0.815003
request                   0.897012
offer                     0.995423
aid_related               0.780038
medical_help              0.921551
medical_products          0.949905
search_and_rescue         0.975969
security                  0.981310
military                  0.969612
child_alone               1.000000
water                     0.956389
food                      0.945582
shelter                   0.936427
clothing                  0.986014
money                     0.979148
missing_people            0.988303
refugees                  0.966561
death                     0.957915
other_aid                 0.869930
infrastructure_related    0.935919
transport                 0.955499
buildings                 0.953083
electricity               0.979021
tools                     0.993897
hospitals                 0.989955
shops                     0.996440
aid_centers               0.986904
other_infrastructure      0.956516
weather_related     

In [38]:
#make columns equal in y_pred and y_test
y_pred = pd.DataFrame(y_pred, columns = y.columns)
y_test = pd.DataFrame(y_test, columns=y.columns)

In [40]:
#replace all 2 with 1
y_pred = y_pred.replace(2,1)

In [41]:
#print classification report
target_names=y.columns
print(classification_report(y_test, y_pred, target_names = target_names))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5992
               request       0.83      0.49      0.62      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.69      0.72      3219
          medical_help       0.67      0.06      0.12       638
      medical_products       0.90      0.06      0.12       418
     search_and_rescue       0.59      0.05      0.10       192
              security       0.00      0.00      0.00       144
              military       0.62      0.07      0.12       245
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.36      0.51       500
                  food       0.85      0.63      0.72       878
               shelter       0.82      0.37      0.51       705
              clothing       0.73      0.07      0.13       115
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [8]:
#replace 2 with 1 to avoid errors
y=y.replace(2,1)
print(np.unique(y))

[0 1]


In [9]:
#verb extraction
class VerbExtractor(BaseEstimator, TransformerMixin):
    def verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
    
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.verb)
        return pd.DataFrame(X_tagged)

In [20]:
#redefining pipeline 
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('verb', VerbExtractor())
    ])),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])
parameters = {
    'clf__estimator__n_estimators':[100,200],
    'features__text_pipeline__vect__max_df':(0.75,1.0)
}


In [10]:
def build_model():
    pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('verb', VerbExtractor())
    ])),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])
    parameters = {
        'clf__estimator__n_estimators':[100,200],
        'features__text_pipeline__vect__max_df':(0.75,1.0)
    }
    #create grid_search
    cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 3)
    
    return cv

In [11]:
def display_results(cv, y_test, y_pred):
    accuracy = (y_pred == y_test).mean()
    target_names=y.columns
    print('Accuracy:', accuracy)
    print(classification_report(y_test, y_pred, target_names = target_names))
    print('\nBest Parameters:', cv.best_params_)

In [12]:
def main():
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    display_results(model, y_test, y_pred)

main()

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.75 
[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.75, score=0.261049921327421, total= 9.9min
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.75 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 13.1min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.75, score=0.2614790444857674, total= 9.9min
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.75 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 26.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=0.75, score=0.2708154506437768, total= 9.9min
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0 
[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0, score=0.2603347160635102, total= 9.9min
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0 
[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0, score=0.26276641396080674, total=10.0min
[CV] clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0 
[CV]  clf__estimator__n_estimators=100, features__text_pipeline__vect__max_df=1.0, score=0.26995708154506437, total=10.0min
[CV] clf__estimator__n_estimators=200, features__text_pipeline__vect__max_df=0.75 
[CV]  clf__estimator__n_estimators=200, features__text_pipeline__vect__max_df=0.75, score=0.26305249606637104, total=15.8min
[CV] clf__estimator__n_estimators=200, features__te

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 197.4min finished


Accuracy: related                   0.817506
request                   0.895881
offer                     0.995042
aid_related               0.786613
medical_help              0.923341
medical_products          0.951754
search_and_rescue         0.975782
security                  0.982838
military                  0.971014
child_alone               1.000000
water                     0.957475
food                      0.944127
shelter                   0.939550
clothing                  0.987605
money                     0.981503
missing_people            0.988749
refugees                  0.967773
death                     0.959573
other_aid                 0.867086
infrastructure_related    0.937071
transport                 0.955568
buildings                 0.951945
electricity               0.977498
tools                     0.993898
hospitals                 0.991228
shops                     0.995805
aid_centers               0.987223
other_infrastructure      0.957285
weather_re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [27]:
model = build_model()
file = 'disaster_response.pkl'
pickle.dump(model, open(file, 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.